In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from transformers import MobileBertTokenizer, TFMobileBertForSequenceClassification

file_path = '/kaggle/input/emotions1/emotions.csv'
dataset = pd.read_csv(file_path)

dataset.dropna(inplace=True)

def clean_text(text):
    text = re.sub(r'\[USERNAME\]', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    text = text.strip()
    return text

dataset['cleaned_text'] = dataset['text'].apply(clean_text)

label_encoder = LabelEncoder()
dataset['label'] = label_encoder.fit_transform(dataset['label'])

print("Sample counts per class before under-sampling:")
print(dataset['label'].value_counts())

def under_sampling(df, label_col):
    min_count = df[label_col].value_counts().min()
    sampled_df = df.groupby(label_col).apply(lambda x: x.sample(min_count)).reset_index(drop=True)
    return sampled_df

dataset = under_sampling(dataset, 'label')

print("\nSample counts per class after under-sampling:")
print(dataset['label'].value_counts())

X_train, X_test, y_train, y_test = train_test_split(dataset['cleaned_text'], dataset['label'], test_size=0.3, random_state=42, stratify=dataset['label'])

2024-06-03 22:06:18.195133: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-03 22:06:18.195228: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-03 22:06:18.357335: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Sample counts per class before under-sampling:
label
1    141067
0    121187
3     57317
4     47712
2     34554
5     14972
Name: count, dtype: int64

Sample counts per class after under-sampling:
label
0    14972
1    14972
2    14972
3    14972
4    14972
5    14972
Name: count, dtype: int64


/tmp/ipykernel_34/723102189.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = df.groupby(label_col).apply(lambda x: x.sample(min_count)).reset_index(drop=True)


In [2]:
model_name = "google/mobilebert-uncased"
tokenizer = MobileBertTokenizer.from_pretrained(model_name)
model = TFMobileBertForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder.classes_))

def encode_texts(texts, tokenizer, max_len=128):
    return tokenizer(
        texts.tolist(),
        max_length=max_len,
        truncation=True,
        padding='max_length',
        return_tensors='tf'
    )

train_encodings = encode_texts(X_train, tokenizer)
test_encodings = encode_texts(X_test, tokenizer)

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train.values
)).shuffle(len(X_train)).batch(16)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test.values
)).batch(16)

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/847 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/164M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMobileBertForSequenceClassification.

Some layers of TFMobileBertForSequenceClassification were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.summary()

In [3]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
model.compile(optimizer=optimizer, 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])

history = model.fit(train_dataset,
                    epochs=7,
                    validation_data=test_dataset)

Epoch 1/7
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


I0000 00:00:1717452867.743334     110 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3931/3931 [==============================] - 1813s 356ms/step - loss: 8720.6875 - accuracy: 0.1871 - val_loss: 1.8273 - val_accuracy: 0.2277
Epoch 2/7
3931/3931 [==============================] - 1339s 341ms/step - loss: 2.4056 - accuracy: 0.2538 - val_loss: 1.5931 - val_accuracy: 0.3314
Epoch 3/7
3931/3931 [==============================] - 1313s 334ms/step - loss: 1.5194 - accuracy: 0.3698 - val_loss: 1.2534 - val_accuracy: 0.5101
Epoch 4/7
3931/3931 [==============================] - 1295s 329ms/step - loss: 0.8822 - accuracy: 0.6677 - val_loss: 0.4620 - val_accuracy: 0.8477
Epoch 5/7
3931/3931 [==============================] - 1293s 329ms/step - loss: 0.3483 - accuracy: 0.8805 - val_loss: 0.2156 - val_accuracy: 0.9233
Epoch 6/7
3931/3931 [==============================] - 1299s 331ms/step - loss: 0.1909 - accuracy: 0.9275 - val_loss: 0.1571 - val_accuracy: 0.9446
Epoch 7/7
3931/3931 [==============================] - 1294s 329ms/step - loss: 0.3936 - accuracy: 0.9418 - val_loss: 0

In [4]:
pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.9 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=c3adfb2bb4d188b78f1476271cec89af8b3041b61cc02808d5c2a4ef62c6fbf2
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
     

In [5]:
from googletrans import Translator

emotion_labels = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}

def translate_texts(texts, src_lang='id', dest_lang='en'):
    translator = Translator()
    translated_texts = []
    for text in texts:
        translation = translator.translate(text, src=src_lang, dest=dest_lang)
        translated_texts.append(translation.text)
        print(f'Translated Text: {translation.text}')  # Menampilkan hasil terjemahan
    return translated_texts

def predict_new_texts(model, tokenizer, texts, label_encoder, max_len=128):
    translated_texts = translate_texts(texts)
    clean_texts = [clean_text(text) for text in translated_texts]

    inputs = tokenizer(clean_texts, max_length=max_len, truncation=True, padding='max_length', return_tensors='tf')

    pred_prob = model.predict(inputs)
    pred_classes = np.argmax(pred_prob.logits, axis=1)
    
    labels = [emotion_labels[pred] for pred in pred_classes]
    return labels

new_texts = [
    "Saya merasa sangat sedih dan kesepian akhir-akhir ini, tidak tahu harus berbuat apa.",
    "Saya sangat marah dan kecewa dengan teman-teman saya.",
    "Hari ini saya merasa sangat bahagia dan penuh semangat!",
    "Saya merasa takut dan cemas setiap kali memikirkan masa depan.",
    "Saya merasa sangat bangga dengan pencapaian saya hari ini.",
    "Saya merasa hampa dan tidak termotivasi untuk melakukan apa pun.",
    "Saya merasa begitu dicintai dan diperhatikan oleh keluarga saya.",
    "Saya merasa kesal dengan situasi di tempat kerja saya.",
    "Saya merasa takut terhadap peluang baru ini.",
    "Saya merasa bersalah dan menyesal tentang kesalahan yang saya buat.",
    "Saya merasa sangat mencintai pasangan saya begitu juga sebaliknya"
]

predicted_labels = predict_new_texts(model, tokenizer, new_texts, label_encoder)
for text, label in zip(new_texts, predicted_labels):
    print(f'Text: {text}\nPredicted Label: {label}\n')


Translated Text: I feel very sad and lonely lately, don't know what to do.
Translated Text: I am very angry and disappointed with my friends.
Translated Text: Today I feel very happy and full of enthusiasm!
Translated Text: I feel scared and anxious every time I think of the future.
Translated Text: I feel very proud of my achievements today.
Translated Text: I feel empty and not motivated to do anything.
Translated Text: I feel so loved and cared for by my family.
Translated Text: I was annoyed with the situation at my workplace.
Translated Text: I feel afraid of this new opportunity.
Translated Text: I feel guilty and sorry about the mistakes I made.
Translated Text: I feel very loving my partner and vice versa
1/1 [==============================] - 40s 40s/step
Text: Saya merasa sangat sedih dan kesepian akhir-akhir ini, tidak tahu harus berbuat apa.
Predicted Label: sadness

Text: Saya sangat marah dan kecewa dengan teman-teman saya.
Predicted Label: anger

Text: Hari ini saya mera

In [6]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

y_pred = model.predict(test_dataset)
y_pred_labels = np.argmax(y_pred.logits, axis=1)

cm = confusion_matrix(y_test, y_pred_labels)

target_names = dataset['label'].unique().astype(str).tolist()

evaluation_report = classification_report(y_test, y_pred_labels, target_names=target_names)

f1 = f1_score(y_test, y_pred_labels, average='weighted')

precision = precision_score(y_test, y_pred_labels, average='weighted')
recall = recall_score(y_test, y_pred_labels, average='weighted')

accuracy = accuracy_score(y_test, y_pred_labels)

print("Confusion Matrix:")
print(cm)

print("\nEvaluation Report:")
print(evaluation_report)

print("\nF1 Score:", f1)
print("Precision:", precision)
print("Sensitivity (Recall):", recall)
print("Accuracy:", accuracy)

1685/1685 [==============================] - 276s 141ms/step
Confusion Matrix:
[[4306   11    3   98   69    4]
 [  18 4060  332   23   15   43]
 [   7   14 4465    6    0    0]
 [  71   13    3 4349   56    0]
 [  31    2    1  199 3884  375]
 [   3    5    2    0    1 4481]]

Evaluation Report:
              precision    recall  f1-score   support

           0       0.97      0.96      0.96      4491
           1       0.99      0.90      0.94      4491
           2       0.93      0.99      0.96      4492
           3       0.93      0.97      0.95      4492
           4       0.96      0.86      0.91      4492
           5       0.91      1.00      0.95      4492

    accuracy                           0.95     26950
   macro avg       0.95      0.95      0.95     26950
weighted avg       0.95      0.95      0.95     26950


F1 Score: 0.9474276140186353
Precision: 0.9496553703799208
Sensitivity (Recall): 0.9478664192949907
Accuracy: 0.9478664192949907


In [8]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

tflite_model_path = 'model_text_mobile.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_quantized_model)
    
from IPython.display import FileLink
FileLink(tflite_model_path)

Summary on the non-converted ops:
---------------------------------
 * Accepted dialects: tfl, builtin, func
 * Non-Converted Ops: 773, Total Ops 5434, % non-converted = 14.23 %
 * 773 ARITH ops

- arith.constant:  773 occurrences  (f32: 752, i32: 21)



  (f32: 748)
  (f32: 48)
  (f32: 1)
  (f32: 1, i32: 289)
  (f32: 3)
  (i32: 1)
  (f32: 362)
  (uq_8: 3, i32: 578)
  (f32: 218)

  (i32: 292)
  (f32: 2)
  (uq_8: 365)
  (i32: 1)
  (i32: 578)
  (f32: 746, i32: 1)
  (i32: 292)
  (f32: 24)
  (f32: 3, i32: 4)
  (f32: 1)
  (f32: 96)


/kaggle/working/model_text_mobile.tflite